# How to build an AFF3CT custom module

In this tutorial you will learn how to build a custom AFF3CT module written in `Python`. The module called `py_modulator` performs BPSK modulation. It has one inputs `b` representing the `N` bits to be modulated and one output `x` which represents the modulated symbols. The full code of the `py_modulator` module is given below.

In [1]:
import numpy as np
import sys  
sys.path.insert(0, '../build/lib')
from py_aff3ct.module.py_module import Py_Module

# Step 1
class py_modulator(Py_Module):

    # Step 2
    def bpsk(self, b, x): # Your original code (independently from aff3ct)
        x[:] = 1.0 - 2.0*b[:]
        return 0
    
    # Step 3
    def __init__(self, N):
        # __init__ (step 3.1)
        Py_Module.__init__(self) # Call the aff3ct Py_Module __init__
        self.name = "py_BPSK"   # Set your module's name

        # __init__ (step 3.2)
        t_mod = self.create_task("modulate") # create a task for your module
        
        # __init__ (step 3.3)
        sb = self.create_socket_in (t_mod, "b", N, np.int32  ) # create an input socket for the task t_mod
        sx = self.create_socket_out(t_mod, "x", N, np.float32) # create an output socket for the task t_mod
    
        # __init__ (step 3.4)
        self.create_codelet(t_mod, lambda slf, lsk, fid: slf.bpsk(lsk[sb], lsk[sx])) # create codelet

In the previous code block we start by importing the minimual packages for building an `py_aff3ct` module.
We need `numpy` for the socket handling, `sys` for adding the path of `py_aff3ct` to the python path.
We also need to import the `Py_Module` module of `py_aff3ct`.
```python
import numpy as np
import sys  
sys.path.insert(0, '../build/lib')
from py_aff3ct.module.py_module import Py_Module
```

**Step 1** Once this import is performed, building a custom `module` for `py_aff3ct` basically consists in writing a Python class heritating from the class `Py_Module`. This explains the `py_modulator` definition
```python
class py_modulator(Py_Module):
```

**Step 2** Write the effective data processing. Here this processing a method of the created class and it corresponds to the following piece of code
```python
def bpsk(self, b, x): # Your original code (independently from aff3ct)
    x[:] = 1.0 - 2.0*b[:]
    return 0
```

* **Remark 1** Here we impose that data passed through sockets are inputs of the function and the function returns an integer.

* **Remark 2** In `aff3ct`, a socket should have a well defined size and type, so we have decided (here) that `b` is a `numpy`array of size `N` and type `np.int32` while `x` is a `numpy`array of size `N` and type `np.float32`. 

**Step 3** Write the `__init__` method. Here the size of the input and output arrays is used for the class initialization.

```python
def __init__(self, N):
```

**Step 3.1**,  since our class inheritates from `Py_Module` the `__init__` method of the `Py_Module` class should be called. `Py_Module` classes have a `name`attribute that we have also set.
```python
    # __init__ (step 3.1)
    Py_Module.__init__(self) # Call the aff3ct Py_Module __init__
    self.name = "py_BPSK"   # Set your module's name
```

**Step 3.2** is to create an empty AFF3CT task and give it a name (here the name of the task is "modulate").
```python
    # __init__ (step 3.2)
    t_mod = self.create_task("modulate") # create a task for your module
```

**Step 3.3** add sockets to the created task. In `aff3ct`, sockets has a given **size** and **type**. For this example we have defined the following sockets:
* Input  socket named `b`, size `N`, data type `np.int32`
* Output socket named `z`, size `N`, data type `np.float32`

The socket declaration is done in the following part of the code:
```python
    # __init__ (step 3.3)
    sb = self.create_socket_in (t_mod, "b", N, np.int32  ) # create an input socket for the task t_mod
    sx = self.create_socket_out(t_mod, "x", N, np.float32) # create an output socket for the task t_mod
```

* **Remark 1**: the name and order of declaration of the socket can be different from the order of the function arguments.

* **Remark 2**: AFF3CT handles the following data types :
    * `np.int8`
    * `np.int16`
    * `np.int32`
    * `np.int64`
    * `np.float32`
    * `np.double`
        
**Step 3.4** "Link" everything by creating a `codelet` for the task `t_add`. A `codelet`is basically a `lambda` function that is attached to a `task` and that has the following signature: 
```
(aff3ct.module.Module, List[aff3ct.module.Socket], Int) -> Int
```

This lambda function is the one called by AFF3CT. Its inputs represent
* an AFF3CT module that can be thought as `self`
* a List of `sockets`, the ones created above in the class definition in the order of creation
* a frame id (not used in this example)

The `codelet` creation is performed by the following line
```python
    # __init__ (step 3.4)
    self.create_codelet(t_mod, lambda slf, lsk, fid: slf.bpsk(lsk[sb], lsk[sx])) # create codelet
```

**Remark**: inputs x, y, z of the method `add` are sockets whereas when first writting the function these inputs could have been `numpy` arrays. The hack is here to use the syntax `[]` for a `socket` object which returns a `numpy` array.


Now that our `module` is created, let's use it !

In [2]:
import py_aff3ct

K   = 8 

# Build the modules
src = py_aff3ct.module.source.Source_random(K)
mod = py_modulator(K)

# Bind the tasks
mod['modulate::b'].bind(src['generate::U_K'])
src('generate').debug = True
mod('modulate').debug = True

# Execute the tasks
src('generate').exec();
mod('modulate').exec();

# Source_random::generate(int32 U_K[8])
# {OUT} U_K = [    0,     1,     1,     0,     1,     1,     1,     1]
# Returned status: [0 'SUCCESS']
#
# py_BPSK::modulate(const int32 b[8], float32 x[8])
# {IN}  b = [    0,     1,     1,     0,     1,     1,     1,     1]
# {OUT} x = [ 1.00, -1.00, -1.00,  1.00, -1.00, -1.00, -1.00, -1.00]
# Returned status: [0 'SUCCESS']
#
